In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Downloads/2008.csv')

In [3]:
df.shape

(7009728, 29)

In [4]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [5]:
pd.DataFrame(df.isnull().sum()/df.shape[0]*100)

,0
Year,0.000000
Month,0.000000
DayofMonth,0.000000
DayOfWeek,0.000000
DepTime,1.943670
CRSDepTime,0.000000
ArrTime,2.163408
CRSArrTime,0.000000
UniqueCarrier,0.000000
FlightNum,0.000000


In [ ]:
df = df.loc[df.Month.isin([1,2,3]),:]

In [ ]:
df.drop(df[df['ArrTime'].isnull()].index, inplace = True)
df.drop(df[df['ActualElapsedTime'].isnull()].index, inplace = True)
df.drop(df[df['CRSElapsedTime'].isnull()].index, inplace = True)
df.drop(df[df['TaxiIn'].isnull()].index, inplace = True)
df.drop(df[df['ArrDelay'].isnull()].index, inplace = True)

In [ ]:
df['CarrierDelay'].fillna((df['CarrierDelay'].min()), inplace=True)
df['WeatherDelay'].fillna((df['WeatherDelay'].min()), inplace=True)
df['NASDelay'].fillna((df['NASDelay'].min()), inplace=True)
df['SecurityDelay'].fillna((df['SecurityDelay'].min()), inplace=True)
df['LateAircraftDelay'].fillna((df['LateAircraftDelay'].min()), inplace=True)

In [ ]:
df.drop[df[df.Cancelled == 1].index, inplace = True]

In [ ]:
df.drop(columns = ['Cancelled', 'Diverted', 'FlightNum', 'TailNum', 'CancellationCode', 'Year'], inplace = True)

In [ ]:
df.shape

In [ ]:
df['Lateness'] = [1 if x >= 30 else 0 for x in df.loc[:,'ArrDelay']]

In [ ]:
df.dtypes

In [ ]:
sns.countplot(x='Month', hue = 'Lateness', data = df)

In [ ]:
plt.figure(figsize = (12,4))
sns.countplot(x='DayofMonth', hue = 'Lateness', data = df)

In [ ]:
sns.countplot(x='DayOfWeek', hue = 'Lateness', data = df)

In [ ]:
df1 = df.sort_values('UniqueCarrier', ascending = False)
sns.countplot(x='UniqueCarrier', hue = 'Lateness', data = df1)

In [ ]:
df2 = df.sort_values('Origin', ascending = False)
sns.countplot(x='Origin', hue = 'Lateness', data = df2)

In [ ]:
df3 = df.sort_values('Dest', ascending = False)
sns.countplot(x='Dest', hue = 'Lateness', data = df3)

Apply one-hot coding for categorical data

In [ ]:
#df = pd.get_dummies(df, columns=['Origin'])
df = pd.get_dummies(df, columns=['UniqueCarrier'])
for column in ['Origin', 'Dest']:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns] = dummies

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import mean_squared_error

In [ ]:
x = df.loc[:, df.columns != 'Lateness']
y = df.Lateness

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 0)

Apply Naive Bayes on the original data

In [ ]:
nb = GaussianNB()
nb.fit(x_train, y_train)
print(classification_report(y_test, nb.predict(x_test)))

Apply Linear Regression

In [ ]:
ln = LinearRegression()
ln.fit(x_train, y_train)
print(classification_report(y_test, ln.predict(x_test)))

Apply Ridge Regression and tuning theta coefficent

In [ ]:
rid = RidgeClassifier(alpha = 0.5, class_weight = 'balanced')
rid.fit(x_train, y_train)
print(classification_report(y_test, rid.predict(x_test)))

In [ ]:
metrics.mean_squared_error(y_test, rid)

In [ ]:
rid = RidgeClassifier(alpha = 1, class_weight = 'balanced')
rid.fit(x_train, y_train)
print(classification_report(y_test, rid.predict(x_test)))

In [ ]:
metrics.mean_squared_error(y_test, rid)

In [ ]:
rid = RidgeClassifier(alpha = 5, class_weight = 'balanced')
rid.fit(x_train, y_train)
print(classification_report(y_test, rid.predict(x_test)))

In [ ]:
metrics.mean_squared_error(y_test, rid)

In [ ]:
Apply SVM

In [ ]:
_svm = svm.LinearSVC()
_svm.fit(x_train, y_train)
print(classification_report(y_test, svm.predict(x_test)))

In [ ]:
_svm = svm.LinearSVC(class_weight = 'balanced')
_svm.fit(x_train, y_train)
print(classification_report(y_test, svm.predict(x_test)))

Apply Logistic Regression

In [ ]:
lr = LogisticRegression(class_weight = 'balanced')
lr.fit(x_train, y_train)
print(classification_report(y_test, lr.predict(x_test)))

Apply Decision Tree

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
print(classification_report(y_test, dt.predict(x_test)))

Apply Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(x_train, y_train)
print(classification_report(y_test, gb.predict(x_test)))

Apply Random Forest

In [ ]:
rfr = RandomForestClassifier()
rfr.fit(x_train, y_train)
print(classification_report(y_test, rfr.predict(x_test)))

Down sampling for imbalanced data

In [ ]:
down_sampling = data[data.Lateness == 1]
down_sampling = down_sampling(resemble(df[df.Lateness == 0], n_samples = df[df.Lateness == 1].value_counts(), random_state = 0)
down_sampling.Lateness.value_counts()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 0)

Apply Naive Bayes on the original data

In [ ]:
nb = GaussianNB()
nb.fit(x_train, y_train)
print(classification_report(y_test, nb.predict(x_test)))

Apply Linear Regression

In [ ]:
ln = LinearRegression()
ln.fit(x_train, y_train)
print(classification_report(y_test, ln.predict(x_test)))

Apply Ridge Regression and tuning theta coefficent

In [ ]:
rid = RidgeClassifier(alpha = 0.5, class_weight = 'balanced')
rid.fit(x_train, y_train)
print(classification_report(y_test, rid.predict(x_test)))

In [ ]:
metrics.mean_squared_error(y_test, rid)

In [ ]:
Apply SVM

In [ ]:
_svm = svm.LinearSVC()
_svm.fit(x_train, y_train)
print(classification_report(y_test, svm.predict(x_test)))

In [ ]:
_svm = svm.LinearSVC(class_weight = 'balanced')
_svm.fit(x_train, y_train)
print(classification_report(y_test, svm.predict(x_test)))

Apply Logistic Regression

In [ ]:
lr = LogisticRegression(class_weight = 'balanced')
lr.fit(x_train, y_train)
print(classification_report(y_test, lr.predict(x_test)))

In [ ]:
Apply Decision Tree

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
print(classification_report(y_test, dt.predict(x_test)))

Apply Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(x_train, y_train)
print(classification_report(y_test, gb.predict(x_test)))

Apply Random Forest

In [ ]:
rfr = RandomForestClassifier()
rfr.fit(x_train, y_train)
print(classification_report(y_test, rfr.predict(x_test)))